# Analysis of Data

Plotting time!!!

## Claim Analysis:
Can refuting fake news make a difference over time?

In [3]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
# Count of real and fake tweets per day sumed over all claims
# Also relative plots

sns.set_theme(style="whitegrid")
ax = sns.barplot(x="Day", y="Frequency", data=tips)
plt.title('Frequency of Real and Fake Tweets for all Claims per Day')

In [ ]:
sns.set_theme(style="whitegrid")
ax = sns.barplot(x="Day", y="Frequency", data=tips)
plt.title('Relative Frequency of Real and Fake Tweets for all Claims per Day')

In [ ]:
# Count of real and fake tweets per claim
# Also relative plots

sns.set_theme(style="whitegrid")
ax = sns.barplot(x="Day", y="Frequency", data=tips)
plt.title('Relative Frequency of Real and Fake Tweets for all Claims per Day')

In [ ]:
# Real and fake tweets average likes over time
# Real and fake tweets average retweets over time

## News Analysis:
Comparing the spread of real and fake news.

In [ ]:
# For the top x real and fake news, compare the relative frequency of posts over time
# For the top x real and fake news, compare the number of likes over time
# For the top x real and fake news, compare the number of retweets over time